# Initial imports cell

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from prophet import Prophet
!pip install neuralprophet[live]
from neuralprophet import NeuralProphet

# Load data

In [ ]:
data = pd.read_csv("AAPL.csv")
df = pd.read_csv('AAPL.csv')
print("Initial data shape:", df.shape)
print("\nFirst few rows:")
print(data.head())

# Check for duplicates and missing values

In [ ]:
print("\nDuplicate rows:", df.duplicated().sum())
print("\nMissing values:")
print(data.isnull().sum())

# Data info function

In [ ]:
def resumetable(df):
    summary = pd.DataFrame({
        'Name': df.columns,
        'dtypes': df.dtypes,
        'Missing': df.isnull().sum(),
        'Uniques': df.nunique()
    })
    print(f"Dataset Shape: {df.shape}")
    return summary

# Display data summary

In [ ]:
print("\nData Summary:")
print(resumetable(df))
print("\nDescriptive Statistics:")
print(df.describe())

# Data preprocessing

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df['Date_ordinal'] = df['Date'].apply(lambda date: date.toordinal())

# Correlation analysis

In [ ]:
numerical_df = df.select_dtypes(include=['number'])
plt.figure(figsize=(16, 8))
sns.heatmap(numerical_df.corr(), cmap="Blues", annot=True)
plt.title("Correlation Heatmap")
plt.show()

# Basic visualizations

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(df['Close'], df['Volume'])
plt.ylabel("Close")
plt.xlabel("Volume")
plt.title("Correlation between Close and Volume")
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(df['High'], df['Low'])
plt.ylabel("High")
plt.xlabel("Low")
plt.title("Correlation between High and Low")
plt.show()

# Candlestick chart

In [ ]:
fig = go.Figure(data=[go.Candlestick(x=data["Date"],
                                    open=data["Open"],
                                    high=data["High"],
                                    low=data["Low"],
                                    close=data["Close"])])
fig.update_layout(title="Apple Stock Price Analysis", xaxis_rangeslider_visible=False)
fig.show()

# Time series analysis

In [ ]:
data = data.set_index(['Date'])
plt.figure(figsize=(20, 10))
plt.plot(data['Adj Close'])
plt.xlabel("Date")
plt.ylabel("Adj Close")
plt.title("Apple Stock Price Over Time")
plt.show()

# Rolling statistics

In [ ]:
window_size = 12
rolling_mean = data.rolling(window=window_size).mean()
rolling_std = data.rolling(window=window_size).std()

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(data['Adj Close'], color='red', label='Actual')
plt.plot(rolling_mean['Adj Close'], color='green', label='Rolling Mean')
plt.plot(rolling_std['Adj Close'], color='black', label='Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show()

# LSTM Model preparation

In [ ]:
y = data['Adj Close']
scaler = MinMaxScaler(feature_range=(0, 1))
y = scaler.fit_transform(np.array(y).reshape(-1, 1))

# Split data

In [ ]:
training_size = int(len(y) * 0.65)
train_data, test_data = y[0:training_size,:], y[training_size:len(y),:1]

# Create sequences

In [ ]:
def create_dataset(dataset, time_step=100):
    X, y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), 0])
        y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(y)

In [ ]:
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape data for LSTM

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# Build LSTM model

In [ ]:
model = Sequential([
    LSTM(100, return_sequences=True, input_shape=(100, 1)),
    Dropout(0.2),
    LSTM(100, return_sequences=True),
    Dropout(0.2),
    LSTM(100, return_sequences=True),
    Dropout(0.2),
    LSTM(100, return_sequences=True),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(1)
])

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, 
                       verbose=1, mode='auto', restore_best_weights=True)

# Train model

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                   callbacks=[monitor], verbose=1, epochs=10)# use 1000 for greater accuracy

# Plot training history

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

# Make predictions

In [ ]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Inverse transform predictions

In [ ]:
train_predict = scaler.inverse_transform(train_predict)
test_predict = scaler.inverse_transform(test_predict)

# Calculate RMSE

In [ ]:
print('Train RMSE:', np.sqrt(mean_squared_error(y_train, train_predict)))
print('Test RMSE:', np.sqrt(mean_squared_error(y_test, test_predict)))

# Plot predictions

In [ ]:
look_back = 100
plt.figure(figsize=(20, 10))
plt.plot(scaler.inverse_transform(y), color='green', label='Actual Stock Price')
plt.plot(train_predict, color='red', label='Training Predictions')
plt.plot(test_predict, color='blue', label='Testing Predictions')
plt.title('Apple Stock Price Prediction')
plt.xlabel('Time Steps')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

# Prophet Model

In [ ]:
df_prophet = df[['Date', 'Adj Close']].rename(columns={'Date': 'ds', 'Adj Close': 'y'})
prophet_model = Prophet(daily_seasonality=True)
prophet_model.fit(df_prophet)

# Make future predictions

In [ ]:
future = prophet_model.make_future_dataframe(periods=58)
forecast = prophet_model.predict(future)

# Plot Prophet predictions

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'],
                        mode='lines', name='Forecast'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'],
                        mode='lines', name='Lower Bound'))
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'],
                        mode='lines', name='Upper Bound'))
fig.add_trace(go.Scatter(x=df_prophet['ds'], y=df_prophet['y'],
                        mode='lines', name='Actual'))
fig.update_layout(title='Prophet Model Forecast', xaxis_title='Date',
                 yaxis_title='Stock Price')
fig.show()

# Neural Prophet Model

In [ ]:
np_model = NeuralProphet()
metrics = np_model.fit(df_prophet, freq='D', epochs=10)# use 1000 for greater accuracy

# Plot Neural Prophet predictions

In [ ]:
forecast = np_model.predict(df_prophet)
np_model.plot(forecast)
plt.title('Neural Prophet Forecast')
plt.show()

# Plot components

In [ ]:
np_model.plot_components(forecast)
plt.show()